In [13]:
import pandas as pd
import geopandas as gpd
import requests
from bs4 import BeautifulSoup
import requests
import json
from datetime import datetime


In [16]:
def get_target_zips(state, city=None, zip_code=None):
    zips = pd.read_csv(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\zipcode_source\zip_code_database.csv")
    
    if city is None and zip_code is None:
        target_zips = zips[zips["state"] == state]["zip"].tolist()
    elif zip_code is None:
        target_zips = zips[(zips["primary_city"] == city) & (zips["state"] == state)]["zip"].tolist()
    else:
        target_zips = [zip_code]
    
    return target_zips


def get_stingray_rgn_id(zip):
    query_location_api = f"https://www.redfin.com/stingray/do/query-location?location={zip}&v=2"
    response = requests.get(query_location_api, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}) 
    soup = BeautifulSoup(response.text, 'html.parser').text
    prefix_removed = soup.split('&&', 1)[1]
    data = json.loads(prefix_removed)
    try:
        region_id = data["payload"]["exactMatch"].get("id").split("_",1)[1]
        return region_id
    except:
        print(f"No Exact match found for zip: {zip}")
        return None


def build_stingray_gis_params(params):
        return "&".join(f"{key}={value}" for key, value in params.items() if params.get(key) != None)


def call_stingray_buy_gis(params_url):
    api_url = "https://www.redfin.com/stingray/api/gis"
    url = f"{api_url}?{params_url}"
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'})
    soup = BeautifulSoup(response.text, 'html.parser').text
    prefix_removed = soup.split('&&', 1)[1]
    # print(url)
    data = json.loads(prefix_removed)

    return data


def parse_stingray_buy_gis(data):
    homes = data.get('payload', {}).get('homes', [])
    parsed_homes = []
    
    for home in homes:
        lat_long = home.get('latLong', {}).get('value', {})
        home_info = {
            "MLS ID": home.get('mlsId', {}).get('value'),
            "Status": home.get('mlsStatus'),
            "Price": home.get('price', {}).get('value'),
            "HOA Fee": home.get('hoa', {}).get('value'),
            "Square Feet": home.get('sqFt', {}).get('value'),
            "Price per Square Foot": home.get('pricePerSqFt', {}).get('value'),
            "Lot Size": home.get('lotSize', {}).get('value'),
            "Bedrooms": home.get('beds'),
            "Bathrooms": home.get('baths'),
            "Location": home.get('location', {}).get('value'),
            "Stories": home.get('stories'),
            "Address": home.get('streetLine', {}).get('value'),
            "City": home.get('city'),
            "State": home.get('state'),
            "ZIP Code": home.get('postalCode', {}).get('value'),
            "Year Built": home.get('yearBuilt', {}).get('value'),
            "URL": home.get('url'),
            "Latitude": lat_long.get('latitude'),
            "Longitude": lat_long.get('longitude')
        }
        parsed_homes.append(home_info)
    
    return parsed_homes


def geocode_dataframe(df, latitude_col='Latitude', longitude_col='Longitude'):
    import geopandas as gpd
    import pandas as pd
    """
    Geocode the given DataFrame based on geographic data files.

    Parameters:
    df (pd.DataFrame): DataFrame containing the data to be geocoded.
    longitude_col (str): Name of the column containing longitude values.
    latitude_col (str): Name of the column containing latitude values.

    Returns:
    pd.DataFrame: Geocoded DataFrame.
    """
    # Convert the DataFrame to a GeoDataFrame
    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df[longitude_col], df[latitude_col]), crs="EPSG:4326"
    )

    # Load and preprocess demographic areas
    demographic_areas = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\census_block_group_source_nationwide\v107\blkgrp.gdb")
    demographic_areas.to_crs("EPSG:4326", inplace=True)
    demographic_areas["GEOID"] = (
        demographic_areas["STATE_FIPS"].astype(str).str.zfill(2)
        + demographic_areas["COUNTY_FIPS"].astype(str).str.zfill(3)
        + demographic_areas["TRACT_FIPS"].astype(str).str.zfill(6)
        + demographic_areas["BLOCKGROUP_FIPS"].astype(str)
    )
    demographic_areas = demographic_areas[["GEOID", "geometry"]].rename(columns={"GEOID": "cbg_geoid"})

    # Load and preprocess CBSA areas
    cbsa_source = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\cbsa_source\tl_2020_us_cbsa.shp")
    cbsa_source.to_crs("EPSG:4326", inplace=True)
    cbsa_source = cbsa_source[["GEOID", "NAME", "geometry"]].rename(columns={"GEOID": "cbsa_geoid", "NAME": "cbsa_name"})

    # Load and preprocess state areas
    state_source = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\state_source\States_shapefile.shp")
    state_source.to_crs("EPSG:4326", inplace=True)
    state_source = state_source[["FID", "State_Code", "geometry"]].rename(columns={"FID": "state_id", "State_Name": "state_name"})

    # Perform spatial joins
    geocoded_dots = gdf.sjoin(demographic_areas, how="left").drop(["index_right"], axis=1)
    geocoded_dots = geocoded_dots.sjoin(cbsa_source, how='left').drop(["index_right"], axis=1)
    geocoded_dots = geocoded_dots.sjoin(state_source, how='left').drop(["index_right"], axis=1)

    # Drop unnecessary columns
    geocoded_dots = geocoded_dots.drop(['geometry'], axis=1)

    return pd.DataFrame(geocoded_dots)


In [14]:
#Load Spatial Datasets
demographic_areas = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\census_block_group_source_nationwide\v107\blkgrp.gdb")
demographic_areas.to_crs("EPSG:4326", inplace=True)

demographic_areas["GEOID"] = \
    demographic_areas["STATE_FIPS"].astype(str).str.zfill(2)  \
    + demographic_areas["COUNTY_FIPS"].astype(str).str.zfill(3) \
    + demographic_areas["TRACT_FIPS"].astype(str).str.zfill(6) \
    + demographic_areas["BLOCKGROUP_FIPS"].astype(str)

demographic_areas = demographic_areas[["GEOID", "geometry"]].rename(columns={"GEOID":"cbg_geoid"})

cbsa_source = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\cbsa_source\tl_2020_us_cbsa.shp")
cbsa_source.to_crs("EPSG:4326", inplace=True)
cbsa_source = cbsa_source[["GEOID", "NAME", "geometry"]].rename(columns={"GEOID":"cbsa_geoid", "NAME": "cbsa_name"})

state_source = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\state_source\States_shapefile.shp")
state_source.to_crs("EPSG:4326", inplace=True)
state_source = state_source[["FID", "State_Code", "geometry"]].rename(columns={"FID":"state_id", "State_Name": "state_name"})

In [18]:
# GIS Search API

import requests

Zip = None
City = None
States = ["WA"]


target_zips = []

for State in States:
    target_zips.extend(get_target_zips(State, City, Zip))

print(f"Number of Zipcodes to be Scrubbed: {len(target_zips)}")

print(target_zips)
data = []

for index, zip in enumerate(target_zips):
    if index % 10 == 0:
        print(f"{index} Zip Codes Evaluated")


    params = {
    #??Active Listings
    "al": 1,
    #Include Nearby Homes
    "include_nearby_homes": "false",
    # Market. ie Seattle
    "market": None,
    # Number of homes to retrieve
    "num_homes": 350,
    #How to Sort the homes
    "ord": "days-on-redfin-asc",
    "page_number": 1,
    "poly": None,
    #Listing Types
    "sf": "1,2,3,4,5,6,7",
    "start": None,
    "status": 9,
    # User input property types (currently only single family, townhomes, multifamily : 134)
    "uipt": "1,3,4",
    # ??API Version?
    "v": 8,
    "zoomLevel": None,
    #Type of Region analyzed
    "region_type" : 2,
    "region_id" : get_stingray_rgn_id(zip)
    }

    if params.get("region_id") == None:
        continue
    else:
        url_param = build_stingray_gis_params(params)
        json_data = call_stingray_buy_gis(url_param)
        list_data = parse_stingray_buy_gis(json_data)
        
        data.extend(list_data)

df= pd.DataFrame(data)

df.drop_duplicates(subset=["MLS ID"], inplace=True)

df["updated_date"] = datetime.now().date()


Number of Zipcodes to be Scrubbed: 733
[98001, 98002, 98003, 98004, 98005, 98006, 98007, 98008, 98009, 98010, 98011, 98012, 98013, 98014, 98015, 98019, 98020, 98021, 98022, 98023, 98024, 98025, 98026, 98027, 98028, 98029, 98030, 98031, 98032, 98033, 98034, 98035, 98036, 98037, 98038, 98039, 98040, 98041, 98042, 98043, 98045, 98046, 98047, 98050, 98051, 98052, 98053, 98054, 98055, 98056, 98057, 98058, 98059, 98061, 98062, 98063, 98064, 98065, 98068, 98070, 98071, 98072, 98073, 98074, 98075, 98077, 98082, 98083, 98087, 98089, 98092, 98093, 98101, 98102, 98103, 98104, 98105, 98106, 98107, 98108, 98109, 98110, 98111, 98112, 98113, 98114, 98115, 98116, 98117, 98118, 98119, 98121, 98122, 98124, 98125, 98126, 98127, 98129, 98131, 98132, 98133, 98134, 98136, 98138, 98139, 98141, 98144, 98145, 98146, 98148, 98151, 98154, 98155, 98158, 98160, 98161, 98164, 98165, 98166, 98168, 98170, 98171, 98174, 98175, 98177, 98178, 98181, 98184, 98185, 98188, 98189, 98190, 98191, 98194, 98195, 98198, 98199, 9

In [19]:
print('Beginning Geocoding')


gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df["Longitude"], df["Latitude"]), crs="EPSG:4326"
)

# Perform spatial joins
geocoded_dots = gdf.sjoin(demographic_areas, how="left").drop(["index_right"], axis=1)
geocoded_dots = geocoded_dots.sjoin(cbsa_source, how='left').drop(["index_right"], axis=1)
geocoded_dots = geocoded_dots.sjoin(state_source, how='left').drop(["index_right"], axis=1)

# Drop unnecessary columns
geocoded_dots = geocoded_dots.drop(['geometry'], axis=1)

homes_geocoded = pd.DataFrame(geocoded_dots)

# display(new_rentals_geocoded)


Beginning Geocoding


In [22]:
print('Beginning Deduping')

existing_homes = pd.read_csv(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Redfin Data\for_sale_homes.csv")



existing_homes['MLS ID'] = existing_homes['MLS ID'].astype(str).str.strip()
homes_geocoded['MLS ID'] = homes_geocoded['MLS ID'].astype(str).str.strip()

# Identify common Property IDs
common_property_ids = existing_homes[existing_homes['MLS ID'].isin(homes_geocoded['MLS ID'])]

# Filter out these common Property IDs from the existing rentals DataFrame
existing_rentals = existing_homes[~existing_homes['MLS ID'].isin(common_property_ids['MLS ID'])]

updated_rentals = pd.concat([existing_rentals, homes_geocoded], ignore_index=True)

print(homes_geocoded.shape[0], " Rentals Downloaded")
print(common_property_ids.shape[0], " Duplicate Rentals")
print(updated_rentals.shape[0], " Total Rentals in Dataset")

Beginning Deduping
16784  Rentals Downloaded
16784  Duplicate Rentals
16784  Total Rentals in Dataset
